# Extended Sources
*In this notebook, I show a first impression of how an extended source will look in Arcus. This is not an exhaustive exploration of all possible geometries and not a complete end-to-end simulation, but a discussion of the general concepts and problems involved.*

In [ ]:
from nbtemplate import display_header, get_path
display_header('ExtendedSources.ipynb', status='Arcus 2020')

## Extended sources
Arcus design goal is spectroscopy from point sources and the mechanical layout will make the observation of extended source challenging:

- Arcus PSF is asymmetric with a bow-tie shape that is much longer in cross-dispersion direction than in dispersion direction. For an image in 0th order that means that the spatial resolution in dispersion direction is much better in one direction than in the other, which makes it hard to interpret any image.
- Arcus has four channels that place the dispersed spectra as (roughly) parallel strips on the cameras. Because of the wings of the PSF and cross-dispersion from the L1 CAT grating support structure, even for point sources there will be some contamination fomr one channel to another. The larger the source, the closer the spectra get. For sources with radii $>30$ arcsec or so, the zero order images and spectra will overlap directly, even without considering the PSF wings or the L1 dispersion.
- In other instruments like Chandra/HETGS the CCD energy resolution might help to assign observed features to a particular grating order, but in Arcus, many grating orders fall on the same location of the camera and they are so closely spaced in energy, that it is often umbiguous to assign a grating order to a photons, even in the absence of cross-talk between different channels or between spatial and spectral information.

## Dispersive spectroscopy of extended sources
Arcus is slitless, so spatial and spectral information is mixed in the dispersed spectrum. Essentially, a photon detected at slightly further distance from the nominal aimpoint could either be a red-shifted emission line emitted at the nominal aimpoint, or a non-redshifted emission line emitted by a region that's located slightlly to the "left" of the nominal aimpoint, or even an extremely red-shifted line from an emission regions that is located to the "right" of the aimpoint, or a superposition of all three scenarios. 

In the case of a point source, we know the location of the emission and can thus retrieve the kinematic information. Conversely, if we know that an extended source has a sharp emission feature that emits as the same location everyhwere in the source, we will simply see image in that line on the detector.

This is not a new problem and has been addressed for e.g. Chandra/HETG observations of supernova remnants, but for Arcus, there are a few twists to it. On the one hand, the size and shape of the PSF (compared to Chandra's sharp, circular PSF) hinders the interpretation of the spatial distribution, on the other hand - at least in principle!-, we can make use of the fact that Arcus has foud channels and sees most lines in several orders
Two channels disperse in one direction and the other two in the opposite direction. That means that spatial and kinematic effects that add up in one channel, will subtract in the opposite channel. Comparing images seen in the lines between the two groups of channels can help to distinguish between spatial and kinematic features. Secondly, we can make use of the fact that we see most lines more than once. Emission that comes from a component close to, but not exactly at the nominal aimpont, will be offset by the same number of pixel in both orders. On the other hand, the grating equation $n\lambda = d \sin\theta$ means that photons of a given $\lambda$ will be offset by a different amount in different orders.

In practice though, we rarely have pure emission line sources with known spatial or kinematic properties. The goal ofthis notebook is to simulate an extended source to get an impression of how the data could look like.

In [ ]:
import os
import numpy as np
from astropy.table import QTable
import astropy.table
import astropy.units as u
from astropy.visualization import (MinMaxInterval, AsymmetricPercentileInterval, SqrtStretch,
                                   ImageNormalize)


from marxs.source.source import DiskSource
from arcus.instrument import Arcus
from arcus.defaults import coords, DefaultPointing, DefaultSource

from sherpa.astro.xspec import XSvapec
import matplotlib.pyplot as plt
from matplotlib import ticker

%matplotlib inline

## Simulations
For this first pass of simulations, I set up a point source, an extended source in the shape of a disk with 30 arcsec radius and a ring (inner radius 25 arcsec, outer radius 30 arcsec). Each of those sources is assigned an APEC spectrum with 2 keV temperature and default abundances. For this study, we simply need an input spectrum with some continuum and a few strong lines in the soft X-ray band; this is not meant to reproduce a specific astrophysical source. The simulation is run for $10^5$ input photons for the point source and $10^6$ photons for the extended sources. Because the signal for the extended source is "smeared out", we simply need more photons to recognize features. 

In [ ]:
vapec = XSvapec()
vapec.kT = 2

en = np.arange(.2, 10., .00002)
fluxdens = vapec(en)

In [ ]:
plt.loglog(en, fluxdens)
plt.xlabel('energy [keV]')
out = plt.ylabel('flux density')
out = plt.title('Input spectrum')

Input spectrum.

In [ ]:
# Input as astropy QTable
tablespectrum = QTable([en * u.keV, fluxdens/ u.s / u.cm**2 / u.keV],
                       names=['energy', 'fluxdensity'])

In [ ]:
# define position and spectrum of source
arc = Arcus()
arc.postprocess_steps = []

In [ ]:
# We keep do not set the flux normalization or Arcus effective area.
# That way, our source will simply generate 1 photon / s.
mysource = DiskSource(coords=coords, energy=tablespectrum, a_outer=30 * u.arcsec)
mysource2 = DiskSource(coords=coords, energy=tablespectrum, a_outer=30 * u.arcsec, a_inner=25 * u.arcsec)
pointsource = DefaultSource(coords=coords, energy=tablespectrum)
mypointing = DefaultPointing(coords=coords)

In [ ]:
pointsrc = pointsource.generate_photons(1e5 * u.s)
pointsrc = mypointing(pointsrc)
pointsrc = arc(pointsrc)

In [ ]:
extendsrc = mysource.generate_photons(1e6 * u.s)
extendsrc = mypointing(extendsrc)
extendsrc = arc(extendsrc)

In [ ]:
extendsrc2 = mysource2.generate_photons(1e6 * u.s)
extendsrc2 = mypointing(extendsrc2)
extendsrc2 = arc(extendsrc2)

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=3,
                         subplot_kw={'aspect': 'equal'}, figsize=(20, 12))

for i, ccdid in enumerate([6, 11, 13]):   
    for j, obs in enumerate([pointsrc, extendsrc, extendsrc2]):
        ind = obs['CCD'] == ccdid + 1
        hist, xedges, yedges = np.histogram2d(obs['detpix_x'][ind], obs['detpix_y'][ind], 
                                              weights=obs['probability'][ind],
                                              bins=[np.arange(0, 2048, 8), 
                                                    np.arange(0, 1024, 8)])

        norm = ImageNormalize(hist, interval=AsymmetricPercentileInterval(0, 99.9))

        cax = axes[i, j].imshow(hist.T, extent=[0, 2048, 0, 1024],
                                origin='lower', aspect='equal', 
                                cmap=plt.get_cmap('inferno'),
                                norm=norm)
        cbar = fig.colorbar(cax, ax=axes[i, j])
        cbar.set_label('counts / bin')

        axes[i, j].set_xlim(0, 2048)
        axes[i, j].set_ylim(0, 1024)
        axes[i, j].set_title(f'CCD: {ccdid + 1}')

The figure shows three of Arcus 16 CCDs (rows) for each of the source geometries (columns: point source, disk, ring). The top row contains the zero order for two of the channels, the other rows the dispersed spectrum on the other camera.

Note that the MARXS code assigns and tracks fractional probabilities to photons, so any one pixel on the detector will receive a non-integer number of photons. The effect of that is that shorter simulations can deliver information on the expected distribution of photons, but it also means that post-processing is needed to capture the full effect of Poisson noise (since MARXS is a Monte-Carlo code, some Poisson scatter is always present even with the probability-tracking trick). In other words, the images should not be interpreted as "this is what Arucs will see with X photons", but "this pattern will emerge if we expose long enough". 
As I said above, the simulations for the extended sources are run with ten times more photons than for the point source. Since the areas of the disk and ring source are different, that means that they do not have the same luminosity per sky area.

The images of the 0 order for the point source (top left) show the bow-tie shape and also small dots above and below the source that are due to L1 dispersion. Since the source is much bigger than the PSF, the shape of the bow-tie is not visible in the disk source, but its effect can be seen in the ring source. The ring is narrower and has a higher surface flux density on the left and right compared to the top and bottom, because the left and right side align with the shape of the PSF, while the ring runs perpendicular to the long axis of the PSF on the top and botton, leading to a more smeared out image. The same effet can be seen in the emission lines (middle right and bottom right). The input source has a constant surface flux density, but because of the shape of the PSF, the image of the source in any emission line appears wider and fainter at the top and bottom of the ring than on the left and right.

In the figure, there are several examples of emission lines. A good case is in the bottom row around $x=400$ and $x=600$ pixel. (Channels are offset from one another such that features to not fall into a chip gap for all channels at the same wavelength.) The lines are sharp as expected for the point source, since the lines input to the simulation are so narrow that they remain unresolved. For the disk and ring source, an image of the source in this line is seen. Because this image stretches over a much larger region, and spatial and spectral information is mixed, the contrast to the source continnum is less than in the point source. Essentially, continuum from more wavelengths and more spatial regions overlaps with the emission line than for the point source.

The middle row (in the channel at $y=800$) there are two closely-spaced emission lines around $x=0$ and $x=100$ that are well-resolved in the point source, but overlap for the extended sources. If those two lines originate from different diffraction orders, it might still be possible to separate them in energy space, but if they are from the same diffraction order, there is no way to assign a photon to one or the other line.

## Conclusion
The simulations above make somewhat idealized assumptions with very simple source geomteries. Real sources will have more spatial structure like knots, jets, or shock fronts. In some sense, those real sources might actually be easier to analyze, because we can concentrate on the bright knots. That said, the simulations above show that about 1 arcmin diameter is a practical limit, because channels begin to overlap. Also, because spatial and kinematic information cannot be disentangled, sources need to have strong emission lines; continuum dominated sources won't work. That said, despite the considerable challenge of data reduction, we can learn things from spatially extended sources with Arcus.